In [1]:
%load_ext autoreload
%autoreload 2

In [94]:
import joblib
import os

In [4]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [5]:
ls

MANIFEST.in                            lexicon/
Makefile                               notebooks/
Procfile                               raw_data/
README.md                              requirements.txt
WorkforceSentimentMonitoring/          scripts/
WorkforceSentimentMonitoring.egg-info/ setup.py
app.py                                 setup.sh
build/                                 test_data/
dist/                                  tests/
joblib_files/


# Redo mnb feature extraction

In [88]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [6]:
# load train test data
from WorkforceSentimentMonitoring.data import get_prepaired_data

In [7]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...


  0%|          | 0/10 [00:00<?, ?it/s]

Dropping initial text columns...
Identifying entries in other languages...


 30%|███       | 3/10 [02:24<05:36, 48.00s/it]

100%|██████████| 10/10 [03:12<00:00, 19.20s/it]


Drop 438 entries? [y] / n
y
Dropping 438 entries...
Splitting train and test...
Encoding targets...
Done!


In [74]:
X_train_cp = X_train.copy()

In [75]:
X_test_cp = X_test.copy()

In [15]:
os_path = os.getcwd()
os_path_abs = '/'.join(os_path.split('/')[:-1])
model_path = os.path.join(os_path_abs,'WorkforceSentimentMonitoring/joblib_files')
model_path

'/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring/joblib_files'

In [65]:
model_nb_carrer = joblib.load(os.path.join(model_path, 'career-opportunities_nb.joblib'))
model_nb_carrer

Pipeline(steps=[('vectorizer',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  TfidfVectorizer(analyzer='char',
                                                                  max_df=0.9794162818234204,
                                                                  min_df=0.017949846778863403,
                                                                  ngram_range=(1,
                                                                               5),
                                                                  stop_words='english'),
                                                  'review')])),
                ('classifier', MultinomialNB(alpha=0.0011498185515295503))])

In [90]:
fea_pipe = joblib.load(os.path.join(model_path, 'final_pipe.joblib'))
fea_pipe

Pipeline(steps=[('preprocessor', Preprocessor()),
                ('emo_extractor', EmotionScoresExtractor()),
                ('scaler', CustomMinMaxScaler()),
                ('engineer', FeatureEngineer())])

In [91]:
last_pipe = Pipeline([
    ('mnb', MultiNBFeaturesExtractor()),
    ('feature_pipe', fea_pipe)
])

In [92]:
last_pipe

Pipeline(steps=[('mnb', MultiNBFeaturesExtractor()),
                ('feature_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())]))])

In [96]:
# export the final feature pipeline
joblib.dump(last_pipe, 'joblib_files/last_pipe_test')

['joblib_files/last_pipe_test']

In [56]:
feature_pipe = joblib.load(os.path.join(model_path, 'feature_pipeline.joblib'))
feature_pipe

Pipeline(steps=[('first_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())])),
                ('mnb', MultiNBFeaturesExtractor())])

In [ ]:
X_train_fea = fea_pipe.fit_transform(X_train_cp)

In [73]:
X_train_cp

,review,work-balance_nb
0,A challenging place but also a good place to l...,0
1,Great Place to Start Your Career Microsoft's c...,0
2,"Over time offen. no coaching. Salary OK,work a...",1
3,"No one listens to their employees Decent pay, ...",0
4,Loved my time there! - standard silicon valley...,1
...,...,...
36658,Great experiece best life ever i mean 9 lakhs ...,0
36659,TRMS fun place to work and so much learning no...,1
36660,"Solid pay, people, and work hours Solid pay, p...",0
36661,Great Experience The people and the intelligen...,1


In [76]:
X_train_feature = get_mnb_features(X_train_cp)

In [85]:
X_train_feature_final = fea_pipe.fit_transform(X_train_feature)

100%|██████████| 36663/36663 [00:39<00:00, 939.30it/s]


In [83]:
X_train_fea_final = get_mnb_features(X_train_fea)

ValueError: Column ordering must be equal for fit and for transform when using the remainder keyword

In [86]:
X_train_feature_final

,review,work-balance_nb,culture-values_nb,career-opportunities_nb,comp-benefits_nb,senior-mgmt_nb,overall_nb,length,anger_score,anticipation_score,disgust_score,fear_score,joy_score,sadness_score,surprise_score,trust_score,subjectivity_review,polarity_review
0,a challenging place but also a good place to l...,0,1,1,1,1,1,0.138713,0.029633,0.386863,0.002665,0.047112,0.448213,0.036494,0.027958,0.371328,0.547872,0.199486
1,great place to start your career microsoft's c...,0,1,1,1,0,0,0.036392,0.039316,0.107264,0.029408,0.121476,0.020235,0.164043,0.033469,0.125077,0.569037,0.280303
2,over time offen no coaching salary ok work a a...,1,1,1,1,0,0,0.006593,0.000000,0.156931,0.000000,0.000000,0.104052,0.000000,0.000000,0.136223,0.295000,0.050000
3,no one listens to their employee decent pay gr...,0,1,1,1,0,1,0.007384,0.000000,0.051016,0.000000,0.000000,0.024837,0.000000,0.000000,0.116708,0.708333,0.483333
4,loved my time there standard silicon valley pe...,1,1,1,1,1,1,0.028481,0.085531,0.259345,0.086294,0.000000,0.403580,0.041743,0.029741,0.177813,0.559320,0.209728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36658,great experiece best life ever i mean lakh per...,0,1,1,1,0,0,0.006857,0.000000,0.000000,0.000000,0.000000,0.078623,0.000000,0.000000,0.000000,0.522917,0.547917
36659,trm fun place to work and so much learning not...,1,1,0,1,1,1,0.002637,0.000000,0.168088,0.000000,0.000000,0.218576,0.000000,0.000000,0.000000,0.200000,0.250000
36660,solid pay people and work hour solid pay peopl...,0,1,1,1,1,1,0.004747,0.000000,0.115616,0.000000,0.062652,0.056287,0.000000,0.000000,0.242719,0.400000,-0.166667
36661,great experience the people and the intelligen...,1,1,0,1,0,0,0.007120,0.010048,0.225995,0.000000,0.138804,0.194488,0.170489,0.000000,0.265587,0.568182,0.228788


# Rerun GridSearch of clf

In [89]:
from WorkforceSentimentMonitoring.trainer import MultiNBFeaturesExtractor
from WorkforceSentimentMonitoring.feature_engineering import get_mnb_features

In [67]:
mnb = MultiNBFeaturesExtractor()

In [44]:
X_train_cp = X_train.copy()
X_test_cp = X_test.copy()

In [45]:
X_test_cp

,review
0,Terrible Culture The benefits offered by the c...
1,Creative The best part of being a Creative it ...
2,"Getting a little bloated, but still a brillian..."
3,Google PRY is not the same as Google Free Snac...
4,"Fulfillment associate Good benefits, lots of o..."
...,...
15709,Amazing products but a tough job One of the be...
15710,Academic and without direction Amazing benefit...
15711,Nothing changed since NYT article .. Very smar...
15712,Not the same company it once was No more pros....


In [71]:
X_train_fea

,review,length,anger_score,anticipation_score,disgust_score,fear_score,joy_score,sadness_score,surprise_score,trust_score,subjectivity_review,polarity_review,work-balance_nb
0,a challenging place but also a good place to l...,0.138713,0.029633,0.386863,0.002665,0.047112,0.448213,0.036494,0.027958,0.371328,0.547872,0.199486,0
1,great place to start your career microsoft's c...,0.036392,0.039316,0.107264,0.029408,0.121476,0.020235,0.164043,0.033469,0.125077,0.569037,0.280303,0
2,over time offen no coaching salary ok work a a...,0.006593,0.000000,0.156931,0.000000,0.000000,0.104052,0.000000,0.000000,0.136223,0.295000,0.050000,1
3,no one listens to their employee decent pay gr...,0.007384,0.000000,0.051016,0.000000,0.000000,0.024837,0.000000,0.000000,0.116708,0.708333,0.483333,0
4,loved my time there standard silicon valley pe...,0.028481,0.085531,0.259345,0.086294,0.000000,0.403580,0.041743,0.029741,0.177813,0.559320,0.209728,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36658,great experiece best life ever i mean lakh per...,0.006857,0.000000,0.000000,0.000000,0.000000,0.078623,0.000000,0.000000,0.000000,0.522917,0.547917,0
36659,trm fun place to work and so much learning not...,0.002637,0.000000,0.168088,0.000000,0.000000,0.218576,0.000000,0.000000,0.000000,0.200000,0.250000,1
36660,solid pay people and work hour solid pay peopl...,0.004747,0.000000,0.115616,0.000000,0.062652,0.056287,0.000000,0.000000,0.242719,0.400000,-0.166667,0
36661,great experience the people and the intelligen...,0.007120,0.010048,0.225995,0.000000,0.138804,0.194488,0.170489,0.000000,0.265587,0.568182,0.228788,1


In [101]:
test_pipe = joblib.load(os.path.join(model_path, 'last_pipe_test'))
test_pipe

Pipeline(steps=[('mnb', MultiNBFeaturesExtractor()),
                ('feature_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())]))])

In [102]:
X_test_cp_final = test_pipe.fit_transform(X_test_cp)

100%|██████████| 15714/15714 [00:17<00:00, 911.32it/s]


In [103]:
X_test_cp_final

,review,work-balance_nb,culture-values_nb,career-opportunities_nb,comp-benefits_nb,senior-mgmt_nb,overall_nb,length,anger_score,anticipation_score,disgust_score,fear_score,joy_score,sadness_score,surprise_score,trust_score,subjectivity_review,polarity_review
0,terrible culture the benefit offered by the co...,0,1,1,1,1,0,0.008011,0.360500,0.000000,0.258632,0.180766,0.277923,0.645509,0.000000,0.000000,0.735000,-0.180000
1,creative the best part of being a creative it ...,0,1,0,1,1,1,0.054072,0.000000,0.068695,0.000000,0.000000,0.044807,0.000000,0.000000,0.000000,0.645455,0.493636
2,getting a little bloated but still a brilliant...,1,0,0,1,0,1,0.062750,0.117350,0.238532,0.230164,0.163897,0.195522,0.106146,0.000000,0.220024,0.687338,0.083279
3,google pry is not the same a google free snack...,0,1,1,1,1,1,0.018692,0.102348,0.330519,0.000000,0.000000,0.167338,0.000000,0.000000,0.204724,0.475000,0.410000
4,fulfillment associate good benefit lot of over...,0,0,0,1,0,0,0.052737,0.060852,0.456553,0.000000,0.108598,0.398607,0.100685,0.023419,0.305375,0.483333,0.202083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15709,amazing product but a tough job one of the bes...,0,1,1,1,1,1,0.056075,0.096922,0.302245,0.009265,0.092344,0.264482,0.065436,0.059815,0.315884,0.648340,0.280267
15710,academic and without direction amazing benefit...,1,1,0,1,0,1,0.032043,0.034758,0.142067,0.000000,0.058806,0.123476,0.000000,0.000000,0.314035,0.421875,0.121875
15711,nothing changed since nyt article very smart a...,0,1,0,1,0,0,0.162216,0.124503,0.294641,0.189822,0.153283,0.180661,0.169771,0.055505,0.267840,0.595940,0.071220
15712,not the same company it once wa no more pro co...,1,1,1,1,0,0,0.015354,0.000000,0.028309,0.000000,0.000000,0.097893,0.124635,0.016187,0.034718,0.525000,0.090000


In [ ]:
X_train_final

In [ ]:
X_test_final

In [18]:
X_train_cp

,review
0,A challenging place but also a good place to l...
1,Great Place to Start Your Career Microsoft's c...
2,"Over time offen. no coaching. Salary OK,work a..."
3,"No one listens to their employees Decent pay, ..."
4,Loved my time there! - standard silicon valley...
...,...
36658,Great experiece best life ever i mean 9 lakhs ...
36659,TRMS fun place to work and so much learning no...
36660,"Solid pay, people, and work hours Solid pay, p..."
36661,Great Experience The people and the intelligen...


In [19]:
X_train_final = feature_pipe.fit_transform(X_train_cp)

100%|██████████| 36663/36663 [00:40<00:00, 908.04it/s]


In [20]:
X_train_final

,review,work-balance_nb,culture-values_nb,career-opportunities_nb,comp-benefits_nb,senior-mgmt_nb,overall_nb
0,a challenging place but also a good place to l...,0,1,1,1,1,1
1,great place to start your career microsoft's c...,0,1,1,1,0,0
2,over time offen no coaching salary ok work a a...,1,1,0,1,0,0
3,no one listens to their employee decent pay gr...,0,1,1,1,0,1
4,loved my time there standard silicon valley pe...,1,1,0,1,1,1
...,...,...,...,...,...,...,...
36658,great experiece best life ever i mean lakh per...,0,1,1,1,1,0
36659,trm fun place to work and so much learning not...,1,1,0,1,1,1
36660,solid pay people and work hour solid pay peopl...,0,1,1,1,1,0
36661,great experience the people and the intelligen...,1,1,0,1,1,0


In [104]:
# search with SGD
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier()
param = {
    'alpha': stats.loguniform(0.00001, 1),
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['l1', 'l2']#,
    #'class_weight': ['balanced', 'None']
         }
sgd_search = RandomizedSearchCV(sgd, param, scoring='accuracy', n_iter=100, cv=10, n_jobs=-1)

In [ ]:
# final test
sgd_final_balance = sgd_search.fit(X_train_final.drop(columns='review'), y_train['work-balance'])
sgd_final_balance.best_score_, \
sgd_final_balance.best_estimator_

In [ ]:
# final test
sgd_final_culture = sgd_search.fit(X_train_final.drop(columns='review'), y_train['culture-values'])
sgd_final_culture.best_score_, \
sgd_final_culture.best_estimator_

In [ ]:
# final test
sgd_final_benefits = sgd_search.fit(X_train_final.drop(columns='review'), y_train['comp-benefits'])
sgd_final_benefits.best_score_, \
sgd_final_benefits.best_estimator_

In [ ]:
# final test
sgd_final_mgmt = sgd_search.fit(X_train_final.drop(columns='review'), y_train['senior-mgmt'])
sgd_final_mgmt.best_score_, \
sgd_final_mgmt.best_estimator_

In [ ]:
# final test
sgd_final_career = sgd_search.fit(X_train_final.drop(columns='review'), y_train['career-opportunities'])
sgd_final_career.best_score_, \
sgd_final_career.best_estimator_

In [ ]:
# final test
sgd_final_overall = sgd_search.fit(X_train_final.drop(columns='review'), y_train['overall'])
sgd_final_overall.best_score_, \
sgd_final_overall.best_estimator_